In [14]:
# ==========================================================
# Lab         : Lab Assignment 1 - Web Scraping
# Name        : SHAZA ADELINA BINTI KHAIRULLAH
# Student ID  : SN0107857
# Name        : NURALEYA BINTI AZIZAN
# Student ID  : IS01084475
# ==========================================================

from bs4 import BeautifulSoup #to parse and extract HTML content
import requests #to send HTTP request to website
import csv
import time
import datetime


# Function to extract reviews from A page
def scrape_page(soup, reviews_list):

    # Find all the review containers
    for review_element in soup.find_all('div', class_='styles_cardWrapper__g8amG styles_show__Z8n7u'):

        # Extract username
        user_element = review_element.find('span',class_='CDS_Typography_appearance-default__68c681 CDS_Typography_prettyStyle__68c681 CDS_Typography_heading-xs__68c681 styles_consumerName__xKr9c')
        user = user_element.text if user_element else 'N/A' #if username not entered

        # Extract review's comment
        comment_element = review_element.find('p', class_='CDS_Typography_appearance-default__68c681 CDS_Typography_prettyStyle__68c681 CDS_Typography_body-l__68c681')
        comment = comment_element.text if comment_element else '' #if comment not filled

        # Extract review date from <time> tag
        date_element = review_element.find('time')
        date = date_element.get('datetime')

        # Store extracted data into list as dictionary
        reviews_list.append({
            'User': user,
            'Date': date,
            'Comment': comment
        })


# the url of the main rent a car page
base_url = 'https://www.trustpilot.com/review/acerentacar.com'

headers = {'User-Agent': 'Mozilla/5.0'}

# Empty list to store all reviews scraped
reviews = []


# Function to scrape Multiple pages
def scrape_all_pages(url):

    page_count = 0   # Counter to limit web scraping to 5 pages only

    #a loop to check if url exists AND page less than 5
    while url and page_count < 5:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract reviews from current page
        scrape_page(soup, reviews)

        # Find "Next page" button
        next_button = soup.find('a', attrs={'name': 'pagination-button-next'})

        # Check if next_button is found before trying to get its 'href' attribute
        if next_button:
            next_link = next_button.get('href')
            url = 'https://www.trustpilot.com' + next_link
        else:
            # If no next button is found, set url to None to terminate the loop
            url = None

        page_count += 1   # Increase page counter


# Call function to start scraping
scrape_all_pages(base_url)


# Store extracted data into CSV file
with open('ACE_Rent_Car_Review.csv', 'w', encoding='utf-8', newline='') as csvfile:

    # Define column headers
    fieldnames = ['User', 'Date', 'Comment']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write header row
    writer.writeheader()

    # Write each review into CSV file
    for review in reviews:
        writer.writerow(review)


print("Reviews have been saved to ACE_Rent_Car_Review.csv")

Reviews have been saved to ACE_Rent_Car_Review.csv
